# Steps to be followed -

## 1. Importing necessary libraries
## 2. Creating S3 bucket
## 3. Mapping train and test data in S3
## 4. Mapping the path of the models in S3

In [32]:
import sagemaker #using sagemaker to install built-in algos - XGB
import boto3 #local python files can also read bucket contents from boto3 (if public access is given)
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session #if we have to create instance, we have to create Session
import numpy as np
from sagemaker.predictor import csv_serializer

In [7]:
bucket_name = 'new-bankapplication'
my_region = boto3.session.Session().region_name #setting region of the instance
print(my_region)

us-west-1


In [8]:
s3 = boto3.resource('s3') #getting access of s3 bucket
s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration={'LocationConstraint':'us-west-1'})

s3.Bucket(name='new-bankapplication')

In [9]:
#set an output path where trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://new-bankapplication/xgboost-as-a-built-in-algo/output


### Downloading the dataset and storing it in S3

In [11]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve('https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv','bank_clean.csv')
    print('Success download')
except Exception as e:
    print('Data load error: ',e)
    
try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Success - download to dataframe')
except Exception as e:
    print('Error : ',e)

Success download
Success - download to dataframe


In [13]:
# Train-Test split
train_data, test_data = np.split(model_data.sample(frac=1,random_state=42), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [17]:
### Saving Train Into Buckets
## training data starts from dependent features
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [29]:
### Saving Test Into Buckets
## test data starts from dependent features
import os
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)],axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('train.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

## Building and Training XGB- inbuilt algorithm

In [21]:
# in-built Algorithm is stored in the form of container/image. We have to pull that image/container in our instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,'xgboost', repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [22]:
container

'746614075791.dkr.ecr.us-west-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'

In [23]:
# initializing hyperparamters - do not do it in here - billing charger might happen
#hyperparamter in local machine and then final parameters here

hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [26]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, #xgb container
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(), #knowing role to access s3 buckets
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path, #output path saved
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600) # last 3 params reduces the billing time

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [27]:
estimator

In [30]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2022-12-08 06:44:52 Starting - Starting the training job...
2022-12-08 06:45:15 Starting - Preparing the instances for trainingProfilerReport-1670481892: InProgress
.........
2022-12-08 06:46:43 Downloading - Downloading input data..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[06:47:00] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter of CSV input is ','
[06:47:00] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/

## Deploy ML mdel

In [31]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
#initial instance - how many parallel instances we want ; specifying type of it

-----!

#### Prediction of the Test Data

In [ ]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
#xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict ; decoding the encoded format
predictions_array = np.fromstring(predictions[1:], sep=',') # turn the prediction into an array
print(predictions_array.shape)

In [35]:
predictions_array

array([0.02587362, 0.02734977, 0.08641617, ..., 0.68751293, 0.04483907,
       0.10314494])

In [36]:
predictions_array

array([0.02587362, 0.02734977, 0.08641617, ..., 0.68751293, 0.04483907,
       0.10314494])

### Plotting confusion matrix

In [37]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.8%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10809)    34% (152)
Purchase        9% (1103)     66% (293) 



### Deleting the endpoints

In [38]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[{'ResponseMetadata': {'RequestId': 'K60QYXPMJVNY6CJ3',
   'HostId': 'o6D/dkH02v/QU3BtBZ+Ebj+0CB9LD1n+z0YPgCSpodZTGsrakHgh5OCnBkvyQVKeFvLUdfzoV+Q=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'o6D/dkH02v/QU3BtBZ+Ebj+0CB9LD1n+z0YPgCSpodZTGsrakHgh5OCnBkvyQVKeFvLUdfzoV+Q=',
    'x-amz-request-id': 'K60QYXPMJVNY6CJ3',
    'date': 'Thu, 08 Dec 2022 07:32:01 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-12-08-06-44-52-207/debug-output/index/000000000/000000000020_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-12-08-06-44-52-207/profiler-output/system/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-12-08-06-44-52-207/rule-output/ProfilerReport-1670481892/profiler-output/profiler-reports/Dataloader.json'},
